# Supply Chain Shipment Pricing Prediction

## Background
In this study, analysis and pricing prediction is applied to supply chain shipment data. Models which will be explored during this analysis are listed and described below:

| Algorithm                                | Definition                                                                                                       | Characteristic                                                                                    |
|------------------------------------------|------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| Gradient boosted trees model (GBT Model) | Each tree is trained to predict and then "correct" for the errors of the previously trained trees                | A set of shallow decision trees trained sequentially.                                             |
| Multiple linear regression (MLR)         | A statistical technique for estimating a predictive target utilizing a linear relationship between two or more predictive factors for one predictive target. | Predicts a dependent variable using multiple independent variables.                               |
| Deep neural network (DNN)                | An artificial neural network consisting of many hidden layers between an input and output layer.                 | This algorithm can model complex nonlinear relationships, and it contains multiple hidden layers. |
| XGBoost regression                       | Extreme gradient boosting acting as an improved algorithm based on the gradient boosting algorithm.              | Excellent efficiency, flexibility, and portability, and it can prevent overfitting.               |
| LightGBM regression                      | Gradient boost-based algorithm that includes two techniques. (1) Gradient based on one-side sampling and (2) exclusive feature bundling. | An ensemble technique that utilizes a leaf-wise tree partitioning method.                         |

### Import Packages

In [1]:
import os
import numpy as np
import pandas as pd
import pandas_bokeh
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import math

In [2]:
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v1.3.0


## Data Collection and Preprocessing

### Data Collection
This data set provides supply chain health commodity shipment and pricing data. Specifically, the data set identifies Antiretroviral (ARV) and HIV lab shipments to supported countries. In addition, the data set provides the commodity pricing and associated supply chain expenses necessary to move the commodities to countries for use. The dataset has similar fields to the Global Fund's Price, Quality and Reporting (PQR) data. PEPFAR and the Global Fund represent the two largest procurers of HIV health commodities. This dataset, when analyzed in conjunction with the PQR data, provides a more complete picture of global spending on specific health commodities. The data are particularly valuable for understanding ranges and trends in pricing as well as volumes delivered by country. The US Government believes this data will help stakeholders make better, data-driven decisions. Care should be taken to consider contextual factors when using the database. Conclusions related to costs associated with moving specific line items or products to specific countries and lead times by product/country will not be accurate.


In [3]:
# Load the data set into Pandas DataFrame
dataset_df = pd.read_csv("Supply_Chain_Shipment_Pricing_Data.csv")

dataset_df.tail()

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10321,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10322,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,17465,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,True,1392,Freight Included in Commodity Cost,134.03
10323,86823,103-ZW-T30,FPQ-15197,SO-50022,DN-4336,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,36639,72911.61,1.99,0.03,"Cipla, Goa, India",False,Weight Captured Separately,Freight Included in Commodity Cost,85.82


### Data Preprocessing

In [4]:
dataset_df_reduced = dataset_df.copy()

dataset_df_reduced['weight (kilograms)'] = pd.to_numeric(dataset_df_reduced['weight (kilograms)'], errors = 'coerce')
dataset_df_reduced.dropna(inplace = True)

dataset_df_reduced['freight cost (usd)'] = pd.to_numeric(dataset_df_reduced['freight cost (usd)'], errors = 'coerce')
dataset_df_reduced.dropna(inplace = True)

In [5]:
# Explore country count data.
CountryCount_raw = dataset_df['country'].value_counts().nlargest(50)
CountryCount_reduced = dataset_df_reduced['country'].value_counts().nlargest(50)


Country count values are represented greatly by South Africa, Nigeria, and Côte d'Ivoire with values over 1000.

In [ ]:
# Describe numerical values before removal section
dataset_df.describe()

In [ ]:
# Describe manufacturing origin.
ManLocs_raw = dataset_df.groupby('manufacturing site').size().nlargest(5)
ManLocs_reduced = dataset_df_reduced.groupby('manufacturing site').size().nlargest(5)